In [ ]:
!pip install keras
!pip install scikit-learn
!pip install tqdm
!pip install numpy
!pip install scipy

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
gossipcop_folder_path = '/content/drive/My Drive/GNN/gossipcop'
politifact_folder_path = '/content/drive/My Drive/GNN/politifact'

# Example: Listing files in the gossipcop folder
import os
gossipcop_files = os.listdir(gossipcop_folder_path)
print(gossipcop_files)  # This will print the names of the files in the gossipcop folder

['raw', 'processed']


In [ ]:
# Install torch_sparse and related PyTorch Geometric packages
!pip install torch-sparse torch-scatter torch-cluster torch-geometric

# After installation, you can now import torch_sparse
from torch_sparse import coalesce

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1035675 sha256=cad5a6460171ae8bb608a65aef231d9947bedc8ef7749e5963677bd381709010
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=d0ce0ad988665290fdac3de6566869f7420598f2d5a5e9a44d4e7f5e88c49db0
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-

data_loader

In [ ]:
file_path = '/content/drive/My Drive/GNN/gossipcop/new_spacy_feature.npz'
print("File exists:", os.path.exists(file_path))

File exists: False


In [ ]:
import os
import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.utils import to_undirected, add_self_loops
from torch_sparse import coalesce
from torch_geometric.io import read_txt_array
import numpy as np
import scipy.sparse as sp
import random

# Function to read a file from disk
def read_file(folder, name, dtype=None):
    path = os.path.join(folder, f'{name}.txt')
    return read_txt_array(path, sep=',', dtype=dtype)

# Function to split data into graph batches
def split(data, batch):
    node_slice = torch.cumsum(torch.from_numpy(np.bincount(batch)), 0)
    node_slice = torch.cat([torch.tensor([0]), node_slice])

    row, _ = data.edge_index
    edge_slice = torch.cumsum(torch.from_numpy(np.bincount(batch[row])), 0)
    edge_slice = torch.cat([torch.tensor([0]), edge_slice])

    data.edge_index -= node_slice[batch[row]].unsqueeze(0)
    data.__num_nodes__ = torch.bincount(batch).tolist()

    slices = {'edge_index': edge_slice}
    for attr in ['x', 'edge_attr', 'y']:
        item = getattr(data, attr, None)
        if item is not None:
            if item.size(0) == batch.size(0):
                slices[attr] = node_slice
            else:
                slices[attr] = torch.arange(0, batch[-1] + 2, dtype=torch.long)
    return data, slices

# Function to read and process graph data
def read_graph_data(folder, feature):
    node_attributes = sp.load_npz(os.path.join(folder, f'new_{feature}_feature.npz'))
    edge_index = read_file(folder, 'A', torch.long).t()
    node_graph_id = np.load(os.path.join(folder, 'node_graph_id.npy'))
    graph_labels = np.load(os.path.join(folder, 'graph_labels.npy'))

    x = torch.from_numpy(node_attributes.todense()).to(torch.float)
    y = torch.from_numpy(graph_labels).to(torch.long)
    _, y = y.unique(sorted=True, return_inverse=True)

    num_nodes = edge_index.max().item() + 1 if x is None else x.size(0)
    edge_index, _ = add_self_loops(edge_index, None)
    edge_index, _ = coalesce(edge_index, None, num_nodes, num_nodes)

    data = Data(x=x, edge_index=edge_index, y=y)
    data, slices = split(data, torch.from_numpy(node_graph_id).to(torch.long))

    return data, slices

# Custom dataset class
class FNNDataset(InMemoryDataset):
    def __init__(self, root, feature='spacy', empty=False, transform=None, pre_transform=None, pre_filter=None):
        self.root = root
        self.feature = feature
        super(FNNDataset, self).__init__(self.root, transform, pre_transform, pre_filter)
        if not empty:
            self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [f'new_{self.feature}_feature.npz', 'A.txt', 'node_graph_id.npy', 'graph_labels.npy']

    @property
    def processed_file_names(self):
        return 'data.pt'

    def download(self):
        # Data should be pre-downloaded in Google Drive
        pass

    def process(self):
        self.data, self.slices = read_graph_data(self.raw_dir, self.feature)
        torch.save((self.data, self.slices), self.processed_paths[0])

# Example of using the dataset
dataset = FNNDataset(root='/content/drive/My Drive/GNN/gossipcop', feature='spacy')


eval_helper

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score, average_precision_score

def eval_deep(log, loader):
    """
    Evaluate the classification performance given mini-batch data.
    """
    data_size = len(loader.dataset.indices)
    batch_size = loader.batch_size
    size_list = [batch_size] * (data_size // batch_size) + [data_size % batch_size] if data_size % batch_size else [batch_size] * (data_size // batch_size)

    assert len(log) == len(size_list)

    # Initialize metrics
    metrics = {'accuracy': 0, 'f1_macro': 0, 'f1_micro': 0, 'precision': 0, 'recall': 0}
    prob_log, label_log = [], []

    # Calculate metrics for each batch
    for (pred_prob, true_labels), size in zip(log, size_list):
        pred_labels = pred_prob.data.cpu().numpy().argmax(axis=1)
        true_labels = true_labels.data.cpu().numpy()

        prob_log.extend(pred_prob.data.cpu().numpy()[:, 1])
        label_log.extend(true_labels)

        metrics['accuracy'] += accuracy_score(true_labels, pred_labels) * size
        metrics['f1_macro'] += f1_score(true_labels, pred_labels, average='macro', zero_division=0) * size
        metrics['f1_micro'] += f1_score(true_labels, pred_labels, average='micro', zero_division=0) * size
        metrics['precision'] += precision_score(true_labels, pred_labels, zero_division=0) * size
        metrics['recall'] += recall_score(true_labels, pred_labels, zero_division=0) * size

    # Normalize by total data size
    for key in metrics:
        metrics[key] /= data_size

    metrics['auc'] = roc_auc_score(label_log, prob_log)
    metrics['ap'] = average_precision_score(label_log, prob_log)

    return metrics


big_cn

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch_scatter import scatter_mean
from torch_geometric.data import Data, Batch
from tqdm import tqdm

# GNN Layer Definition
class GNNLayer(torch.nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GNNLayer, self).__init__()
        self.conv = GCNConv(in_feats, out_feats)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        return F.relu(x), x

# Bi-Directional GCN Model
class BiGCN(torch.nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super(BiGCN, self).__init__()
        self.TD_layer = GNNLayer(in_feats, hid_feats)
        self.BU_layer = GNNLayer(in_feats, hid_feats)
        self.fc = torch.nn.Linear(hid_feats * 4, out_feats)

    def forward(self, data):
        td_x, td_x_copy = self.TD_layer(data.x, data.edge_index)
        bu_x, bu_x_copy = self.BU_layer(data.x, data.edge_index)

        td_x = torch.cat((td_x, td_x_copy), 1)
        bu_x = torch.cat((bu_x, bu_x_copy), 1)
        x = torch.cat((td_x, bu_x), 1)
        x = scatter_mean(x, data.batch, dim=0)
        return F.log_softmax(self.fc(x), dim=1)

# Sample Dataset
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, size=100):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        x = torch.randn(16, 3)  # Node features
        edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
        return Data(x=x, edge_index=edge_index)

# Parameters
class Args:
    batch_size = 128
    epochs = 45
    nhid = 128
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

args = Args()

# Dataset and DataLoader Setup
dataset = SampleDataset()
num_training = int(len(dataset) * 0.8)
num_test = len(dataset) - num_training
training_set, test_set = random_split(dataset, [num_training, num_test])

train_loader = DataLoader(training_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False)

# Model Setup
model = BiGCN(in_feats=3, hid_feats=args.nhid, out_feats=2)
model = model.to(args.device)

# Training Loop
model.train()
for epoch in tqdm(range(args.epochs)):
    for batch in train_loader:
        batch = batch.to(args.device)
        out = model(batch)
        # Add your loss and optimizer steps here

# Note: Add your loss function, optimizer, and evaluation steps according to your task.


100%|██████████| 45/45 [00:00<00:00, 78.40it/s]


gcnfn

In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GATConv, global_mean_pool, Linear
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from tqdm import tqdm

# Sample Dataset
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, size=100, num_features=310, num_classes=2):
        self.size = size
        self.num_features = num_features
        self.num_classes = num_classes

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        data = torch_geometric.data.Data()
        data.x = torch.randn(16, self.num_features)  # Node features
        data.edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)  # Edge indices
        data.y = torch.randint(0, self.num_classes, (1,))  # Label
        return data

# GCNFN Model
class GCNFN(torch.nn.Module):
    def __init__(self, num_features, nhid, num_classes, concat=False):
        super(GCNFN, self).__init__()
        self.concat = concat
        self.conv1 = GATConv(num_features, nhid * 2)
        self.conv2 = GATConv(nhid * 2, nhid * 2)

        if concat:
            self.fc0 = Linear(num_features, nhid)
            self.fc1 = Linear(nhid * 2 + nhid, nhid)  # Adjusted input size after concatenation
        else:
            self.fc1 = Linear(nhid * 2, nhid)  # Only nhid * 2 from conv2

        self.fc2 = Linear(nhid, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.selu(self.conv1(x, edge_index))
        x = F.selu(self.conv2(x, edge_index))
        x = F.selu(global_mean_pool(x, batch))

        if self.concat:
            news = torch.stack([torch.mean(data.x[data.batch == idx], dim=0) for idx in torch.unique(data.batch)])
            news = F.relu(self.fc0(news))
            x = torch.cat([x, news], dim=1)

        x = F.selu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.log_softmax(self.fc2(x), dim=-1)

        return x

# Settings
num_features = 310
nhid = 128
num_classes = 2
batch_size = 128
epochs = 60
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset and DataLoader
dataset = SampleDataset(num_features=num_features, num_classes=num_classes)
train_set, test_set = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Model
model = GCNFN(num_features, nhid, num_classes, concat=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

# Training Loop
model.train()
for epoch in tqdm(range(epochs)):
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
for data in test_loader:
    data = data.to(device)
    out = model(data)
    pred = out.argmax(dim=1)
    correct += int((pred == data.y).sum())
accuracy = correct / len(test_set)
print(f'Test Accuracy: {accuracy:.4f}')


100%|██████████| 60/60 [00:03<00:00, 16.37it/s]

Test Accuracy: 0.4500


gnncl

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.utils.data import random_split
from torch_geometric.data import DataLoader, Data
import torch_geometric
from tqdm import tqdm

# Corrected Sample Dataset Class
class SampleDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root, size=100, features=3, classes=2, transform=None, pre_transform=None):
        super(SampleDataset, self).__init__(root, transform, pre_transform)
        self.size = size
        self.features = features  # Renamed attribute
        self.classes = classes    # Renamed attribute
        self.data_list = self.generate_data()

    def generate_data(self):
        data_list = []
        for _ in range(self.size):
            data = Data()
            data.x = torch.randn(10, self.features)  # Node features
            data.edge_index = torch.randint(0, 10, (2, 20))  # Edges
            data.y = torch.randint(0, self.classes, (1,))  # Graph label
            data_list.append(data)
        return data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

# Define the GNN Model
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, data.batch)  # Global pooling

        return F.log_softmax(x, dim=1)

# Settings
features = 3   # Number of node features
classes = 2    # Number of classes
hidden_channels = 16

# Dataset and DataLoader
dataset = SampleDataset(root='/tmp/SampleDataset', features=features, classes=classes)
train_set, test_set = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(features, hidden_channels, classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training Loop
model.train()
for epoch in tqdm(range(10)):  # Set the number of epochs
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
for data in test_loader:
    data = data.to(device)
    out = model(data)
    pred = out.argmax(dim=1)
    correct += int((pred == data.y).sum())
accuracy = correct / len(test_set)
print(f'Test Accuracy: {accuracy:.4f}')


100%|██████████| 10/10 [00:00<00:00, 110.90it/s]

Test Accuracy: 0.5500


gnn

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, global_max_pool as gmp
from torch_geometric.data import Data, DataLoader
from torch.utils.data import random_split
from tqdm import tqdm

# Sample Dataset Class (replace with your actual dataset)
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, size=100, num_features=3, num_classes=2):
        self.size = size
        self.num_features = num_features
        self.num_classes = num_classes

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        data = Data()
        data.x = torch.randn(10, self.num_features)  # Node features
        data.edge_index = torch.randint(0, 10, (2, 20))  # Edges
        data.y = torch.randint(0, self.num_classes, (1,))  # Graph label
        return data

# Define the Model
class Model(torch.nn.Module):
    def __init__(self, num_features, nhid, num_classes, model_type='gcn', concat=False):
        super(Model, self).__init__()
        self.num_features = num_features  # Define num_features as an instance attribute
        self.concat = concat

        if model_type == 'gcn':
            self.conv1 = GCNConv(num_features, nhid)
        elif model_type == 'sage':
            self.conv1 = SAGEConv(num_features, nhid)
        elif model_type == 'gat':
            self.conv1 = GATConv(num_features, nhid)

        if concat:
            self.lin0 = torch.nn.Linear(num_features, nhid)
            self.lin1 = torch.nn.Linear(nhid + nhid, nhid)
        else:
            self.lin1 = torch.nn.Linear(nhid, nhid)

        self.lin2 = torch.nn.Linear(nhid, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = gmp(x, data.batch)  # Global max pooling

        if self.concat:
            news = torch.cat([data.x[data.batch == idx][0] for idx in torch.unique(data.batch)], dim=0)
            news = news.view(-1, self.num_features)  # Reshape to match features
            news = F.relu(self.lin0(news))
            x = torch.cat([x, news], dim=1)
            x = F.relu(self.lin1(x))

        x = F.log_softmax(self.lin2(x), dim=-1)

        return x

# Settings
num_features = 3
nhid = 128
num_classes = 2
batch_size = 128
epochs = 35
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset and DataLoader
dataset = SampleDataset(num_features=num_features, num_classes=num_classes)
train_set, test_set = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Model
model = Model(num_features, nhid, num_classes, model_type='sage', concat=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training and Evaluation
model.train()
for epoch in tqdm(range(epochs)):
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

model.eval()
correct = 0
for data in test_loader:
    data = data.to(device)
    out = model(data)
    pred = out.argmax(dim=1)
    correct += int((pred == data.y).sum())
accuracy = correct / len(test_set)
print(f'Test Accuracy: {accuracy:.4f}')


100%|██████████| 35/35 [00:00<00:00, 47.95it/s]

Test Accuracy: 0.5000
